<a href="https://colab.research.google.com/github/touchvarit/Data-Analyst-Portfolio/blob/main/CVXPY_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="8">What is CVXPY?</font>


This workbook is based on https://www.cvxpy.org/index.html

In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


CVXPY is an open source Python-embedded modeling language for convex optimization problems. It automatically

1.   Transforms the problem written in a mathematical (intuitive) expression into standard form for a solver.
2.   Choose the solver suitable for the problem.
2.   Determine all parameters for the solver.
3.   Unpacks the results

#Linear Programming

##Example
Consider the following linear programming problem:\
\
\begin{align*}
\text{minimize: } & 10x+15y+25z,& \\
\text{subject to: } & x+y+z&\geq 1000, \\
 & x-2y&\geq 0, \\
 &z&\geq 340,\\
 &x+z&= 800,\\
 &x,y,z&\geq 0,\\
\end{align*}
where $x,y,z \in \mathbb{R}$

##Solving the Problem Using Standard Solvers




First, consider the **standard form** of a linear optimization problem:

\\
\begin{align*}
\text{minimize:} & \quad c^T x, \\
\text{subject to:} & \quad A_{ub} x \leq b_{ub}, \\
 & \quad A_{eq} x = b_{eq}, \\
\end{align*}
\
where

$A_{ub} \in  \mathbb{R}^{m \times n},b_{ub} \in \mathbb{R}^m,A_{eq} \in  \mathbb{R}^{s \times n},b_{eq} \in \mathbb{R}^s \text{ and } c \in \mathbb{R}^n \text{ are problem data and } x \in  \mathbb{R}^n \text{ is the optimization variable.}$

To solve the above problem, you would need to

1.   Rewrite the problem in the standard form.
2.   Determine c, $A_{ub}$, $b_{ub}$, $A_{eq}$, $b_{eq}$ according to the problem.
2.   Use linear programming solver such as scipy.optimize.linprog(c, $A_{ub}$, $b_{ub}$, $A_{eq}$, $b_{eq}$).



\begin{align*}
\text{minimize: } & 10x+15y+25z,& \\
\text{subject to: }  &-x-y-z &\leq -1000, \\
 & -x+2y+0z &\leq 0, \\
 &0x+0z-z &\leq -340,\\
 &-x+0y+0z &\leq 0,\\
 &0x-y+0z &\leq 0,\\
 &0x+0y-z &\leq 0,\\
 &x+0y+z &= 800,\\
\end{align*}
where $x,y,z \in \mathbb{R}$

\begin{align*}
\text{minimize: } & \begin{bmatrix} 10  \\ 15\\25  \end{bmatrix}^T\begin{bmatrix} x  \\ y\\z  \end{bmatrix} \\
\text{subject to: } & \begin{bmatrix} -1 & -1 &-1 \\ -1 & 2&0\\0&0&-1\\-1&0&0\\0&-1&0\\0&0&-1 \end{bmatrix} \begin{bmatrix} x  \\ y\\z  \end{bmatrix} \leq \begin{bmatrix} -1000  \\ 0\\-340 \\0\\0\\0 \end{bmatrix}, \\
 & \begin{bmatrix} 1&0&1\end{bmatrix}\begin{bmatrix} x  \\ y\\z  \end{bmatrix}=[800] \\
\end{align*}
where $x,y \in \mathbb{R}$


Thus, we have that

$$c= \begin{bmatrix} 10  \\ 15\\25  \end{bmatrix},
A_{ub}=  \begin{bmatrix} -1 & -1 &-1 \\ -1 & 2&0\\0&0&-1\\-1&0&0\\0&-1&0\\0&0&-1 \end{bmatrix} ,
b_{uq}=\begin{bmatrix} -1000  \\ 0\\-340 \\0\\0\\0 \end{bmatrix},
 A_{eq}= \begin{bmatrix} 1&0&1\end{bmatrix},
 b_{eq}=[800] \\
$$

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Setting the coefficients of the linear objective function vector
c = np.array([10, 15, 25])

# Setting the inequality constraints matrix
A_ub = np.array([[-1, -1, -1],
              [-1, 2, 0],
              [0, 0, -1],
              [-1, 0, 0],
              [0, -1, 0],
              [0, 0, -1]])

# Setting the inequality constraints vector
b_ub = np.array([-1000, 0, -340, 0, 0, 0])


# Setting the equality constraints matrix
A_eq = np.array([[1, 0, 1]])

# Setting the equality constraints vector
b_eq = np.array([800])

# Solving linear programming problem
res = linprog(c, A_ub, b_ub,A_eq,b_eq)

print('Optimal value:', round(res.fun, ndigits=2),
      '\nx values:', res.x,
      '\nNumber of iterations performed:', res.nit,
      '\nStatus:', res.message)

Optimal value: 16100.0 
x values: [460. 200. 340.] 
Number of iterations performed: 0 
Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)


##Solving the Problem Using *CVXPY*

Alternativly, one can use CVXPY to solve the problem:

\begin{align*}
\text{minimize: } & 10x+15y+25z,& \\
\text{subject to: } & x+y+z&\geq 1000, \\
 & x-2y&\geq 0, \\
 &z&\geq 340,\\
 &x+z&= 700,\\
 &x,y,z&\geq 0,\\
\end{align*}
where $x,y,z \in \mathbb{R}$.

In [ ]:
#Install CVXPY package
!pip install cvxpy

In [ ]:
import cvxpy as cp
import numpy as np

# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()
z = cp.Variable()

# Form objective.
obj = cp.Minimize(10*x+15*y+25*z)

# Create constraints.
constraints = [x + y + z >= 1000,
               x - 2*y   >= 0,
               z         >= 340,
               x + z     == 800,
               x         >= 0,
               y         >= 0,
               z         >= 0]

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()
# Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var",'x is', x.value,'y is', y.value,'z is', z.value )

status: optimal
optimal value 16100.000003703022
optimal var x is 460.00000025950794 y is 200.0000005063855 z is 339.9999997404864


##Example: Investment Optimization Problem

You have 12,000 to invest, and three different funds from which to choose. The municipal bond fund has a 7% return, the local bank's CDs have an 8% return, and the high-risk account has an expected (hoped for) 12% return. To minimize the risk, you decide not to invest any more than $2,000 in the high-risk account. For tax reasons, you need to invest at least three times as much in the municipal bonds as in the bank CDs. Assuming the year-end yields are expected what are the optimal investment amounts?

**Objective Function**


The objective is to maximize the total return:
$ \text{Maximize} \quad 0.07x_1 + 0.08x_2 + 0.12x_3 $

where:
- $x_1$: amount invested in municipal bonds
- $x_2$: amount invested in local bank's CDs
- $x_3$: amount invested in the high-risk account

**Constraints**
1. Total investment is limited to $12,000:

   $x_1 + x_2 + x_3 \leq 12,000$

2. The amount invested in the high-risk account is limited to $2,000:

   $x_3 \leq 2,000$

3. For tax reasons, invest at least three times as much in municipal bonds as in bank CDs:

   $x_1 \geq 3x_2$

In [ ]:
# Define variables
x1 = cp.Variable()  # Investment in municipal bonds
x2 = cp.Variable()  # Investment in bank CDs
x3 = cp.Variable()  # Investment in high-risk account

# Define parameters
total_budget = 12000
high_risk_limit = 2000

# Objective function
objective = cp.Maximize(0.07 * x1 + 0.08 * x2 + 0.12 * x3)

# Constraints
constraints = [
    x1 + x2 + x3 <= total_budget,
    x3 <= high_risk_limit,
    x1 >= 3 * x2
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal Investment Amounts:")
print("Investment in Municipal Bonds :", x1.value)
print("Investment in Bank CDs :", x2.value)
print("Investment in High-Risk Account :", x3.value)


Optimal Investment Amounts:
Investment in Municipal Bonds : 7500.000000326279
Investment in Bank CDs : 2499.999999695155
Investment in High-Risk Account : 1999.999999955671


## Example: Loan Allocation Optimization Problem

A financial institution has a lending capacity of $1,000,000 and considers three borrower types: A, B, and C with expected returns of 8%, 10%, and 12%, respectively. The institution must allocate loans to all types, with minimums of 30% to A, and 20% to C, and a maximum of 40% to B. The goal is to maximize the total expected return.

**Objective Function**


The objective is to maximize the total expected return:
$ \text{Maximize} \quad 0.08x_A + 0.10x_B + 0.12x_C $

where:
- $x_A$: amount allocated to borrower type A
- $x_B$: amount allocated to borrower type B
- $x_C$: amount allocated to borrower type C

**Constraints**
1. Total loan allocation is limited to $1,000,000:

   $x_A + x_B + x_C = 1,000,000$

2. Minimum allocation constraints:
   - $x_A \geq 0.30 \times 1,000,000$ (30% to A)
   - $x_C \geq 0.20 \times 1,000,000$ (20% to C)

3. Maximum allocation constraint:
   - $x_B \leq 0.40 \times 1,000,000$ (40% to B)

In [ ]:
import cvxpy as cp

# Define variables
xA = cp.Variable()  # Loan allocation to borrower type A
xB = cp.Variable()  # Loan allocation to borrower type B
xC = cp.Variable()  # Loan allocation to borrower type C

# Define parameters
total_capacity = 1000000

# Objective function
objective = cp.Maximize(0.08 * xA + 0.10 * xB + 0.12 * xC)

# Constraints
constraints = [
    xA + xB + xC == total_capacity,
    xA >= 0.30 * total_capacity,
    xC >= 0.20 * total_capacity,
    xB <= 0.40 * total_capacity,
    xA >= 0,
    xB >= 0,
    xC >= 0
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal Loan Allocations:")
print("Loan to Borrower A :", xA.value)
print("Loan to Borrower B :", xB.value)
print("Loan to Borrower C :", xC.value)
print("Total Expected Return:", problem.value)


Optimal Loan Allocations:
Loan to Borrower A : 300000.0000006506
Loan to Borrower B : 5.192941456057914e-05
Loan to Borrower C : 699999.9999474199
Total Expected Return: 107999.99999893537


##Example: Currency Exposure Hedging Optimization Problem

A company has a currency exposure of 100,000 units in USD and EUR. It can hedge using Forward Contracts (0.02 per unit) and Options (0.03 per unit). The goal is to minimize the total hedging cost while meeting the following constraints:

- Total exposure must be fully hedged.
- Forward Contracts usage cannot exceed 70,000 units.
- At least 30,000 units must be hedged using Options.

**Objective Function**


The objective is to minimize the total hedging cost:
$ \text{Minimize} \quad 0.02x_{\text{Forward}} + 0.03x_{\text{Options}} $

where:
- $x_{\text{Forward}}$: units hedged using Forward Contracts
- $x_{\text{Options}}$: units hedged using Options

**Constraints**
1. Total exposure must be fully hedged:

   $x_{\text{Forward}} + x_{\text{Options}} = 100,000$

2. Forward Contracts usage cannot exceed 70,000 units:

   $x_{\text{Forward}} \leq 70,000$

3. At least 30,000 units must be hedged using Options:

   $x_{\text{Options}} \geq 30,000$


In [ ]:
# Define variables
x_forward = cp.Variable()  # Units hedged using Forward Contracts
x_options = cp.Variable()  # Units hedged using Options

# Define parameters
forward_cost_per_unit = 0.02
options_cost_per_unit = 0.03
total_exposure = 100000
forward_contract_limit = 70000
options_minimum = 30000

# Objective function
objective = cp.Minimize(forward_cost_per_unit * x_forward + options_cost_per_unit * x_options)

# Constraints
constraints = [
    x_forward + x_options == total_exposure,
    x_forward <= forward_contract_limit,
    x_options >= options_minimum
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal Hedging Strategy:")
print("Units Hedged using Forward:", x_forward.value)
print("Units Hedged using Options:", x_options.value)
print("Total Hedging Cost:", problem.value)

Optimal Hedging Strategy:
Units Hedged using Forward: 69999.99999996346
Units Hedged using Options: 30000.00000003655
Total Hedging Cost: 2300.0000000003656


#Quadratic Programming

##Example
Minimize the following (quadratic) objective function:


$ f(x, y) = 2x^2 + 5y^2 - 4xy + 6x + 8y $

##Subject to the constraint:
$ 3x + 2y \leq 18 $



##Solving the Problem Using Standard Solvers

To solve the above problem, we need to rewrite it in the **standard form**:

\\
\begin{align*}
\text{minimize:}\quad & \frac{1}{2}x^TPx+q^Tx,& \\
\text{subject to:}\quad & Gx &\leq h&, \\
 & Ax &= b \\
\end{align*}
\
where

$P \in  \mathbb{R}^{n \times n}, q \in \mathbb{R}^n, G \in \mathbb{R}^{m \times n}, h \in \mathbb{R}^m, A \in \mathbb{R}^{p \times n} \text{ and } b \in \mathbb{R}^n \text{ are problem data and } x \in \mathbb{R}^n \text{ is the optimization variable}$

The above problem can be written in the standard form as:

\begin{align*}
\text{minimize:}\quad & \frac{1}{2}\begin{bmatrix} x \\ y \end{bmatrix}^T\begin{bmatrix} 4&-4 \\ -4& 10 \end{bmatrix}\begin{bmatrix} x \\ y \end{bmatrix}+\begin{bmatrix} 6 \\ 8 \end{bmatrix}^T\begin{bmatrix} x \\ y \end{bmatrix}&\\
\text{subject to:}\quad &  \begin{bmatrix} 3&2 \end{bmatrix}\begin{bmatrix} x \\ y \end{bmatrix}\leq [18] &\\
\end{align*}

which can be used using standard solvers such as **scipy**. However, to make our life easier, we will use CVXPY instead.

##Solving the Problem Using *CVXPY*

Please note that the multiplication of two variables is not DCP convex. Thus we need to rearrange the objective function to avoid non DCP convex error. The rest is straightforward.

In [ ]:
# Define the variables
x = cp.Variable()
y = cp.Variable()

# Define the objective function to minimize
#objective = cp.Minimize(2*(x**2) + 5*(y**2) -4*(x*y) + 6*x + 8*y)
# Note that we cannot write xy in CVXPY
objective = cp.Minimize((x-2*y)**2 + x**2 + y**2       + 6*x + 8*y)

# Define the constraint
constraint = [3 * x + 2 * y <= 18]

# Create the optimization problem
problem = cp.Problem(objective, constraint)

# Solve the problem
problem.solve()

# Display the results
print("Optimal x:", x.value)
print("Optimal y:", y.value)
print("Minimum value of the objective function:", problem.value)


Optimal x: -3.8333333333333335
Optimal y: -2.3333333333333335
Minimum value of the objective function: -20.833333333333332


In [ ]:
# @title
# Define the variables
x = cp.Variable()
y = cp.Variable()

# Define the objective function to minimize
Q = np.array([[4, -4], [-4, 10]])
c = np.array([6, 8])
objective = cp.Minimize(0.5 * cp.quad_form(cp.vstack([x, y]), Q) + c.T @ cp.vstack([x, y]))

# Define the constraint
constraint = [3 * x + 2 * y <= 18]

# Create the optimization problem
problem = cp.Problem(objective, constraint)

# Solve the problem
problem.solve()

# Display the results
print("Optimal x:", x.value)
print("Optimal y:", y.value)
print("Minimum value of the objective function:", problem.value)

Optimal x: -3.8333333333333326
Optimal y: -2.333333333333334
Minimum value of the objective function: -20.833333333333336


##Example

\begin{align*}
\text{minimize: } & (x-y)^2, \\
\text{subject to: } & x+y=2, \\
 & x-y \geq 1, \\
\end{align*}
where $x,y \in \mathbb{R}$

In [ ]:
import cvxpy as cp
import numpy as np

# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

# Form objective.
obj = cp.Minimize((x - y)**2)

# Create two constraints.
constraints = [x + y == 2,
               x - y >= 1]

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()
# Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var",'x is', x.value,'y is', y.value)

status: optimal
optimal value 1.0
optimal var x is 1.5 y is 0.5


##Example: Adversing Problem

A company has a total advertising budget of $1,000,000 and wants to allocate this budget across three marketing channels: online advertising, television advertising, and print advertising. The goal is to maximize the overall brand exposure and engagement, which is measured by a quadratic utility function representing the impact of advertising spending on each channel.

The quadratic utility functions for each channel are as follows:

1. **Online Advertising:** $ U_{\text{online}}(x_{\text{online}}) = -0.001x_{\text{online}}^2 + 500x_{\text{online}} $
2. **Television Advertising:** $ U_{\text{TV}}(x_{\text{TV}}) = -0.0005x_{\text{TV}}^2 + 300x_{\text{TV}} $
3. **Print Advertising:** $ U_{\text{print}}(x_{\text{print}}) = -0.0007x_{\text{print}}^2 + 400x_{\text{print}} $

where $x_{\text{online}}$, $x_{\text{TV}}$, and $x_{\text{print}}$ are the amounts spent on online, TV, and print advertising, respectively.

The total budget constraint is:

$ x_{\text{online}} + x_{\text{TV}} + x_{\text{print}} = 1,000,000 $

**Objective Function**

The objective is to maximize the overall quadratic utility:

$ \text{Maximize} \quad U(x) = U_{\text{online}}(x_{\text{online}}) + U_{\text{TV}}(x_{\text{TV}}) + U_{\text{print}}(x_{\text{print}}) $

**Constraints**

Subject to the total budget constraint:

$ x_{\text{online}} + x_{\text{TV}} + x_{\text{print}} = 1,000,000 $



In [ ]:
# Define variables
x_online = cp.Variable()
x_tv = cp.Variable()
x_print = cp.Variable()

# Define parameters
budget = 1000000

# Define quadratic utility functions
U_online = -0.001 * x_online**2 + 500 * x_online
U_tv = -0.0005 * x_tv**2 + 300 * x_tv
U_print = -0.0007 * x_print**2 + 400 * x_print

# Objective function to maximize overall utility
objective = cp.Maximize(U_online + U_tv + U_print)

# Total budget constraint
budget_constraint = [x_online + x_tv + x_print == budget]

# Formulate the problem
problem = cp.Problem(objective, budget_constraint)

# Solve the problem
problem.solve()

# Display the results
print("Optimal solution:")
print("Online Advertising Budget:", x_online.value)
print("Television Advertising Budget:", x_tv.value)
print("Print Advertising Budget:", x_print.value)


Optimal solution:
Online Advertising Budget: 287096.7741928672
Television Advertising Budget: 374193.548386395
Print Advertising Budget: 338709.6774186611


#Solving Other Types of Convex Optimization Problems

##Why Convex Function?

A function $f(x)$ is convex on an interval $[a,b]$ if for any two points $x_1$ and $x_2$ in $[a,b]$ and any $\alpha$ where $0<\alpha<1$,
$$f[\alpha x_1+(1-\alpha) x_2]\leq \alpha f[x_1]+(1-\alpha )f[x_2].$$

In [ ]:
# @title
#กรณีที่ทุกคนเน็ตเร็วนะครับผมหาวิธีไม่ได้ครับผมเจอแต่วิธีที่ต้องผ่าน drive ครับ
from IPython.display import Image, display

# Replace "image_url" with the direct URL of your image
image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c7/ConvexFunction.svg"

# Display the image
display(Image(url=image_url))


If an objective function is convex, we can be sure that there is only one optimal point.

##Disciplined Convex Programming

Disciplined convex programming (DCP) is a system for constructing mathematical expressions with known curvature from a given library of base functions. CVXPY uses DCP to ensure that the specified optimization problems are convex.

This section of the tutorial explains the rules of DCP and how they are applied by CVXPY.

Visit [link text](https://www.cvxpy.org/tutorial/dcp/index.html) (https://www.cvxpy.org/tutorial/dcp/index.html)

##More Atomic Functions

This section of the tutorial describes the atomic functions that can be applied to CVXPY expressions. CVXPY uses the function information in this section and the DCP rules to mark expressions with a sign and curvature.

Visit: [link text](https://www.cvxpy.org/tutorial/dcp/index.html) (https://www.cvxpy.org/tutorial/functions/index.html)

##Example


\begin{align*}
\text{Minimize: } & e^{-(x+y)}, \\
\text{Subject to: } & x + y = 2, \\
 & x - y \geq 1,
\end{align*}


In [ ]:
import cvxpy as cp

# Define variables
x = cp.Variable()
y = cp.Variable()

# Define the objective function
objective = cp.Minimize(cp.exp(-(x + y)))

# Define the constraints
constraints = [x + y == 2, x - y >= 1]

# Formulate the optimization problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal value (minimum):", problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)


Optimal value (minimum): 0.1353352832828886
Optimal x: 2.3252201831737245
Optimal y: -0.3252201835156598


##Example

\begin{align*}
\text{Minimize: } & -\log(x) - 2\log(y), \\
\text{Subject to: } & x + y = 1, \\
& x, y > 0.
\end{align*}


In [ ]:
# Define variables
x = cp.Variable(pos=True)  # x > 0
y = cp.Variable(pos=True)  # y > 0

# Define the objective function
objective = cp.Minimize(-cp.log(x) - 2 * cp.log(y))

# Define the constraint
constraint = [x + y == 1]

# Formulate the optimization problem
problem = cp.Problem(objective, constraint)

# Solve the problem
problem.solve()

# Display the results
print("Optimal value (minimum):", problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)

Optimal value (minimum): 1.9095425051468993
Optimal x: 0.333332119577795
Optimal y: 0.6666678803380326
